# Long Put Option

### I. Strategy Overview

#### Payoff

![](long_put.png)

#### Features:
* Short Delta: option value increases as the underlying price goes down
* Short Theta: option value decreases as time decays
* Long volatility: option value increases as the implied volatility hikes

### II. Strategy Setup

In [1]:
import sys  
sys.path.insert(0, 'C:\\Users\\chena\\Desktop\\Trading\\StrategicOptions\\')
import datetime
from Functions.Backtest import OptionBackTesting, BackTestingVisualization

####  1. Input symbol of underlying

In [2]:
Ticker = 'MSFT'

#### 2. Specify testing period and find out the expiration dates within

In [3]:
Start_Date = datetime.datetime(2017, 1, 4)
End_Date = datetime.datetime(2018,3, 1)

expiry = OptionBackTesting(Ticker, [], [], Start_Date, End_Date,  [], []).set_expiration_calendar()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1088: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


#### 3. Select an expiration date

In [4]:
expiration = datetime.datetime(2018, 3, 18)

#### 4. Choose option enter and exit dates

In [5]:
entry_date = datetime.datetime(2017, 1, 4)
exit_date = datetime.datetime(2018, 3, 1)

#### 5. Option Type & Strike Price Selection
* Change the strike price to see how it affects the strategy

In [6]:
option_type = 'P'
action = 'L'
strike_price = 55  # At the money
# strike_price = 70  # Deep in the money
# strike_price = 65  # Far out of the money

### III. Visualization
#### 1. Generate options basic data

In [9]:
option_data = OptionBackTesting(Ticker, strike_price, expiration, entry_date, exit_date, option_type, action)
option_price, option_return, underlying_price, underlying_return, trading_days, iv, hv = option_data.generate_data()

Long Date
2017-01-04    1.000000e+00
2017-01-05    9.043263e-01
2017-01-06    6.804579e-01
2017-01-09    8.407135e-01
2017-01-10    1.018998e+00
                  ...     
2018-02-23    1.199056e-34
2018-02-26    3.063318e-43
2018-02-27    1.693695e-30
2018-02-28    2.635384e-33
2018-03-01    1.278540e-22
Length: 291, dtype: float64


##### 2. Speficy spot price range and generate payoff chart

In [ ]:
lower_limit = 40 # Modify to customize the lower bound of x-axis
upper_limit = 80 # Modify to customize the upper bound of x-axis

# Generate payoffs
payoff_calc = BackTestingVisualization(trading_days, option_price, underlying_price, option_return, underlying_return, iv, strike_price, expiration, entry_date, exit_date, lower_limit, upper_limit, option_type, action)
payoff_calc.plot_payoff()

# Probability of profit


#### 3. Price Moves

In [ ]:
# viz = BackTestingVisualization(trading_days, option_price, underlying_price, option_return, underlying_return, iv,[],[],[],[],[],[])
payoff_calc.plot_price_history()

#### 4. Returns

In [ ]:
payoff_calc.plot_option_pnl()

### IV. Trade Journaling

In [ ]:
entry_date.date()

In [ ]:
payoff_calc.print_strategy_summary()
payoff_calc.strategy_summary

### V. Recurring Strategy Setup
Get in when:
Simplified
* 1st trading day of every month
* Buy next month's expiration

Realistic:
* IV < 0
* Expiration date < 60 & Expiration date > 60
* VIX < 20

In [ ]:
import pandas as pd
trading_days['Year-Month'] = trading_days['Date'].dt.to_period('M') + 1  # +1 to find the expiration date in the next month
dte_seeker = pd.merge(trading_days, expiry, on='Year-Month', how='left')
dte_seeker['DTE'] = (pd.to_datetime(dte_seeker['Trading Day']) - pd.to_datetime(dte_seeker['Date'])).dt.days
dte_seeker.dropna(inplace=True)

In [ ]:
trading_days['Date'].dt.date

In [ ]:
from datetime import timedelta
entry_dates = dte_seeker[dte_seeker['DTE']==43]['Date'].reset_index(drop=True)
expiration_dates = dte_seeker[dte_seeker['DTE']==43]['Trading Day'].reset_index(drop=True)
exit_dates = entry_dates + timedelta(days=22)

# exit_dates = dte_seeker[(dte_seeker['DTE']==21)]['Date']
# exit_dates[exit_dates>entry_dates[0]].reset_index(drop=True)

entry_dates

In [ ]:
expiration_dates

In [ ]:
exit_dates

In [ ]:
import pandas as pd
expiration_dates = pd.to_datetime(expiration_dates)

In [ ]:
strategy_performance = pd.DataFrame([])
i = 0
strike_price = 55
for _ in entry_dates:
    option_data = OptionBackTesting(Ticker, strike_price, expiration_dates[i], entry_dates[i], exit_dates[i], option_type)
    option_price, option_return, underlying_price, underlying_return, trading_days, iv, hv = option_data.generate_data()
    payoff_calc = BackTestingVisualization(trading_days, option_price, underlying_price, option_return, underlying_return, iv, strike_price, expiration_dates[i], entry_dates[i], exit_dates[i], lower_limit, upper_limit, 'C')
    payoff_calc.print_strategy_summary()
    strategy_performance = strategy_performance.append(payoff_calc.strategy_summary)
    i+=1
    strike_price = underlying_price['close'][-1]+5

In [ ]:
win_rate = "{:.2%}".format(strategy_performance['Win'].sum()/len(strategy_performance['Win']))
# Total_Cost = "${:,.2f}".format(strategy_performance['Cost Basis'].sum())

In [ ]:
win_rate

In [ ]:
strategy_performance